<a href="https://colab.research.google.com/github/aashnasoni25/AI-Project/blob/main/Identification_of_Novel_Epigenomic_%26_Transcriptomic_Cancer_Subtype_Biomarkers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discovering methylomic & transcriptomic biomarkers for cancer subtype classification

Aashna Soni

(run in VSCode)

In [ ]:
#import necessary libraries

import pandas as pd
import numpy as np
import requests
from sklearn.utils import resample
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

DNA Methylation Workflow

Data processing

In [ ]:
def impute_data(df_train, df_test):

    '''
    impute the training dataframe & the corresponding test dataframe using median imputation
    @df_train: training dataset
    @df_test: test dataset
    '''

    first_column_train = df_train.iloc[:, 0].copy()
    last_column_train = df_train.iloc[:, -1].copy()

    first_column_test = df_test.iloc[:, 0].copy()
    last_column_test = df_test.iloc[:, -1].copy()

    # drop the first & last columns for both dataframes before doing median imputation
    df_train = df_train.iloc[:, 1:-1]
    df_test = df_test.iloc[:, 1:-1]

    # combine the training & test dataframes to find the overall median for each feature (column)
    df_combined = pd.concat([df_train, df_test])

    # threshold to remove values - columns that have more than 99% of values missing
    na_counts = df_train.isna().sum()
    threshold = int(0.01 * df_train.shape[0])

    # drop columns with more than 'threshold' missing values from the training & test dataframes
    df_train.dropna(axis='columns', thresh=df_train.shape[0]-threshold, inplace=True)
    df_test = df_test[df_train.columns]

    # fill missing values in remaining columns with the median of the columns
    df_train.fillna(df_train.median(), inplace = True)
    df_test.fillna(df_combined.median(), inplace=True)

    #add back the first and last columns
    df_train.insert(0, "Sample ID", first_column_train)
    df_train.insert(df_train.shape[1], "Cancer Subtype", last_column_train)

    df_test.insert(0, "Sample ID", first_column_test)
    df_test.insert(df_test.shape[1], "Cancer Subtype", last_column_test)

    return df_train, df_test

ANOVA

In [ ]:
def get_cancer_subtype_lists(X_bootstrap, y_bootstrap):

  '''
  separate the bootstrapped training data by cancer subtype before conducting ANOVA
  @X_bootstrap: training dataset (number of rows = number of training examples, number of columns = number of features)
  @Y_bootstrap: corresponding labels for training data
  '''

  y_bootstrap_values = y_bootstrap.values

  #create separate list for each cancer subtype, where the length of the list is the number of training examples with the given cancer subtype label
  bronchus_and_lung = []
  breast = []
  kidney = []
  brain = []
  thyroid_gland = []
  prostate_gland = []

  for i in range(len(y_bootstrap_values)):
    if y_bootstrap_values[index] == "Bronchus and lung":
      bronchus_and_lung.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Breast":
      breast.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Kidney":
      kidney.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Brain":
      brain.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Thyroid gland":
      thyroid_gland.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Prostate gland":
      prostate_gland.append(X_bootstrap.iloc[index].values[1:])

  #convert to numpy arrays

  bronchus_and_lung = np.array(bronchus_and_lung)
  breast = np.array(breast)
  kidney = np.array(kidney)
  brain = np.array(brain)
  thyroid_gland = np.array(thyroid_gland)
  prostate_gland = np.array(prostate_gland)

  return bronchus_and_lung, breast, kidney, brain, thyroid_gland, prostate_gland

In [ ]:
def anova_single_run(X_bootstrap, y_bootstrap):

  '''
  conduct a single run of ANOVA (i.e. ANOVA on a given X_bootstrap & corresponding y_bootstrap)
  @X_bootstrap: training dataset (number of rows = number of training examples, number of columns = number of features)
  @Y_bootstrap: corresponding labels for training data
  '''

  #separate dataset by cancer subtype
  bronchus_and_lung, breast, kidney, brain, thyroid_gland, prostate_gland = get_cancer_subtype_lists(X_bootstrap, y_bootstrap)

  all_features_p_vals = {}
  all_features_f_vals = {}

  col_index = bronchus_and_lung.shape[1] - 1

  #for each feature, find the f-value & p-value
  while col_index >= 0:
    group_labels = ['Bronchus and lung'] * bronchus_and_lung.shape[0] + ['Breast'] * breast.shape[0] + ['Kidney'] * kidney.shape[0] + ['Brain'] * brain.shape[0] + ['Thyroid gland'] * thyroid_gland.shape[0] + ['Prostate gland'] * prostate_gland.shape[0]
    values = []

    values.extend(bronchus_and_lung[:, col_index])
    values.extend(breast[:, col_index])
    values.extend(kidney[:, col_index])
    values.extend(brain[:, col_index])
    values.extend(thyroid_gland[:, col_index])
    values.extend(prostate_gland[:, col_index])

    df = pd.DataFrame({'group': group_labels, 'value': values})

    model = ols('value ~ C(group)', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=3)

    #get f-value & p-value from ANOVA table
    f_value = anova_table['F']['C(group)']
    p_value = anova_table['PR(>F)']['C(group)']

    all_features_p_vals[X_bootstrap.columns[col_index+1]] = p_value
    all_features_f_vals[X_bootstrap.columns[col_index+1]] = f_value

    col_index -= 1

  return all_features_p_vals, all_features_f_vals

In [ ]:
# perform ANOVA on all the bootstrapping round (3 times)
def full_anova(X, y):

  '''
  perform ANOVA 3 times (i.e. on each round of bootstrapping)
  @X: training dataset (number of rows = number of training examples, number of columns = number of features)
  @y: corresponding training dataset labels
  '''

  #each row is 1 run of ANOVA (on 1 bootstrapped dataset). Each column is a feature.
  all_runs_p_vals = {}
  all_runs_f_vals = {}

  feature_names = X.columns.tolist()[1:]

  for feature in feature_names:
    all_runs_p_vals[feature] = 0
    all_runs_f_vals[feature] = 0

  for i in range(3):

    #resample the training dataset (i.e. implement 1 round of bootstrapping)
    X_bootstrap, y_bootstrap = resample(X, y, replace=True)

    #run ANOVA on the bootstrapped dataset
    dict_temp_p, dict_temp_f = anova_single_run(X_bootstrap, y_bootstrap)
    for key in dict_temp_p:
      all_runs_p_vals[key] += dict_temp_p[key]
      all_runs_f_vals[key] += dict_temp_f[key]

  #take the average p-value & f-value across the boostrapping runs
  for key in all_runs_p_vals:
    all_runs_p_vals[key] /= 3.0
    all_runs_f_vals[key] /= 3.0

  return all_runs_p_vals, all_runs_f_vals

In [ ]:
#based on raw data analysis, remove these files since they are duplicates or undesired tissue sample (i.e. not primary tumor tissue, recurrent tumor tissue, metastatic tissue)
files_to_not_include = ['92993161-98c6-4c89-bd0f-b5c6880d1989', 'db568f33-b97c-47af-ac71-3df06e4cc036', 'ed8d1b68-7bf9-431a-a6dd-ddca9a6d16a9', '309280e5-3172-4aec-a321-e2660c46ca61', 'f9807081-f3ab-46fb-947a-3793e12aaf70', 'c26cf2b9-683f-441f-8f94-a4786ef82c2b', '707131c7-060a-41af-82d9-e4914caef2fd', '33ccaa5e-dde9-4001-811b-66da6426aab1', '85935ada-a41c-454d-a690-3a4e1224e6cd', '2286ad74-1a59-4caa-85b7-f06362fed76f', 'a8723a1c-5cac-4521-859c-bd85800a8aab', 'e5d3a051-c890-4478-9d9a-268fc0c18163', '45dfc4d1-7513-4054-bb5b-4ed60ac31a7f', '03f76c75-3da2-4ae3-9b34-c5a328dc226d', 'a61bab4c-b126-48af-a5bd-93486549628a', '3b6a17e1-3460-45b1-b1ce-4339115eb383', 'df1cc485-8133-4c61-a13a-6edcf3eaf31b', '429c4fab-4a86-4288-a51c-051e08280aed', '8a220737-632b-4d1a-a77e-360eb2209f77', 'a7300965-b08e-404c-a0dd-7f9d29c8059f', '4af85a08-7a78-4eb0-965d-8c1b947ed8d6', '9079d799-d028-41a9-a438-af4130fd639c', '16e9a3a7-4baf-45b7-bdf0-a91bb81d247c', '85e19953-eae5-44a5-b6fd-82f13b5739b4', '17121b34-23b1-4213-bd79-3cc2e88b1178', 'dbc50065-31b1-42ce-ab81-7e8388befdfa', '957224a2-5ace-475a-b23a-d81c771119c9', '0c3f625f-5251-4eaa-b32a-7159dd4cc1bf', '72bfc1e2-946b-4dbf-aa0a-17af89574b62', 'cce27565-9751-4393-bff6-0689cf02f6ec', '06217548-9512-470a-b45b-2a69c9bdc276', '1e19d819-a67f-4a43-84aa-58958a1dd421', '513d9acf-5369-4d37-bb8f-df6d5f8dace5', 'abe4666b-dd7f-4f74-bd60-b2ded6fc5968', '45305bde-4558-4ec2-acc5-615a0382e1d3', '7daf5136-253e-4720-9623-1ce470d561d1', 'b2352f52-1558-405c-96d6-4be46eb87fce', 'f32abde4-ee27-48c9-ba48-ffd95fd42258', 'd4f8b411-278e-49c6-b5a3-9c8fddcff3ae', '3342e582-88f1-4faf-b0fd-c5c2fe812396', 'dc2a5f4a-0d98-4858-aed9-533ecebe2758', '9088686d-edf8-4425-9dbb-7a2b51a5e812', 'cacab017-38e8-4a9a-8766-79f0f2374f05', '69188777-b7de-4c0d-b6f3-c03d9bcdcd8a', '8b3d3959-b9a6-40a3-866a-4a4af73c2857', '9d8ca9e8-ecbd-446b-8f27-ed8cead4caec', '1bddc024-a623-4331-a67e-5e34a3135deb', 'b38b01e0-a14c-49c3-893a-33cc111cb4bc', '98e58937-3dc8-468b-870d-0af138fa271f', 'b62d148f-9233-4c06-b299-2c0974637641', 'ff9424c7-f7a0-4de5-9793-249b63caef1c', '80a7a8c7-5d5e-4d11-aab9-75096944ce67', '0873cc0e-9419-42b2-9728-65679d50b0e9', 'ef9c0cab-dcf1-4044-8038-b0d8482beea3', 'd0f6ae77-e832-4244-8562-a6e2721756e1', 'c4a09534-b468-438c-9d80-3f9f7084bf59', '224971c5-d3f4-42c5-9244-7cf7cd3b54c3', 'a4a7eb51-0610-42c4-847f-87a4279de171', '8c3fa390-2665-4720-a48f-f2da41e21057', '31e5af1b-6667-4ce9-8239-f22ea99ce8c1', '604f9109-dea7-4e25-bc06-8e39273ce6f9', '44f9b93e-e16c-48b2-a2fd-88fa11e164d5', 'd8b7c38d-a5aa-4ab0-9d48-dae62e4d530c', '4ac181ef-70c1-41fa-a51b-c0a31d54814d', 'e3e6e591-aad9-4fce-85d4-4b3ffa1210d3', '4a12f7a2-b1bc-40fa-ae64-470c6e1b73de', 'b685e46d-2484-456f-b2a7-7a580a1ef87c', 'f58c8e5e-2031-48a6-a602-2d5c509e488d', '9b466502-c222-4140-a372-7970871b039d', '20e69189-bfc5-426e-b079-b49191516717', '768a8a08-957c-473c-bcde-1fd11e4f447a', 'a0529346-ff59-4a67-a9cf-ff4cec6d3450', 'c1486f85-7c79-456a-ad4d-c8d282c163d1', '4a8fd508-3a9a-4d58-8730-feed56b35f5e', 'fca34ad2-b8eb-4527-b277-ff4603026093', 'f5efd31a-2de2-483a-8713-48c68743742a', '585c3c48-44c8-4c51-840e-763fa3ab1844', '145f8668-e4fe-45f0-a192-7e78b6b64827', 'b2532703-0c56-4a19-aaca-462fa943dd20', '8f867e04-8a13-4b68-9ae6-46ebc2683b4f', 'e57f586b-f7b8-4ae9-89a2-b6636cb0cd04', 'f1f6d9d5-d7d2-4f48-84ff-4a636d1caf0a', '4afe28c0-184f-4531-8256-c749642e03cf', 'c4d91357-9c79-4976-bce3-be8ee363e9e2', '7f8a6fe2-27a5-4603-970f-87b84b899336', '6dc45ff3-754b-4a46-ad70-e795639cec00', '5c98717c-a390-408b-adb9-c0d7eb069794', '3e3adf67-dcdb-4072-8869-8e53a45736d5', '4aa87de9-be18-476c-a1b1-3c7add2ca9d2', 'a26dff9e-05c4-4e74-bd5b-c21a0aa19405', '7f2e4674-f696-4b1d-a002-4f87e3dfd159', '44306bb2-f8a7-4053-8419-09cec65edef6']

In [ ]:
#read in the training data files
df_train_0 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_0.csv")
df_train_1 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_1.csv")
df_train_2 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_2.csv")
df_train_3 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_3.csv")
df_train_4 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_4.csv")
df_train_5 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_5.csv")
df_train_6 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_6.csv")
df_train_7 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_7.csv")
df_train_8 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_8.csv")
df_train_9 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_9.csv")
df_train_10 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_10.csv")
df_train_11 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_11.csv")
df_train_12 = pd.read_csv("C:\\Users\\tsoni\\Downloads\\full_CORRECTED_training_data_12.csv")

In [ ]:
#read in the test data file
df_test_0 = pd.read_csv("C:\\Users\\tsoni\Downloads\\full_CORRECTED_test_data_0.csv")

In [ ]:
#remove unwanted training examples

mask0 = ~df_train_0['Sample ID'].isin(files_to_not_include)
mask1 = ~df_train_1['Sample ID'].isin(files_to_not_include)
mask2 = ~df_train_2['Sample ID'].isin(files_to_not_include)
mask3 = ~df_train_3['Sample ID'].isin(files_to_not_include)
mask4 = ~df_train_4['Sample ID'].isin(files_to_not_include)
mask5 = ~df_train_5['Sample ID'].isin(files_to_not_include)
mask6 = ~df_train_6['Sample ID'].isin(files_to_not_include)
mask7 = ~df_train_7['Sample ID'].isin(files_to_not_include)
mask8 = ~df_train_8['Sample ID'].isin(files_to_not_include)
mask9 = ~df_train_9['Sample ID'].isin(files_to_not_include)
mask10 = ~df_train_10['Sample ID'].isin(files_to_not_include)
mask11 = ~df_train_11['Sample ID'].isin(files_to_not_include)
mask12 = ~df_train_12['Sample ID'].isin(files_to_not_include)

df_train_0 = df_train_0[mask0]
df_train_1 = df_train_1[mask1]
df_train_2 = df_train_2[mask2]
df_train_3 = df_train_3[mask3]
df_train_4 = df_train_4[mask4]
df_train_5 = df_train_5[mask5]
df_train_6 = df_train_6[mask6]
df_train_7 = df_train_7[mask7]
df_train_8 = df_train_8[mask8]
df_train_9 = df_train_9[mask9]
df_train_10 = df_train_10[mask10]
df_train_11 = df_train_11[mask11]
df_train_12 = df_train_12[mask12]

mask_test_0 = ~df_test_0['Sample ID'].isin(files_to_not_include)
df_test_0 = df_test_0[mask_test_0]

In [ ]:
#impute each training data file & perform ANOVA

df_train_1_imputed = impute_data(df_train_1)
anova1_p, anova1_f = full_anova(df_train_1_imputed.iloc[:, :-1], df_train_1_imputed.iloc[:, -1])


df_train_2_imputed = impute_data(df_train_2)
anova2_p, anova2_f = full_anova(df_train_2_imputed.iloc[:, :-1], df_train_2_imputed.iloc[:, -1])


df_train_3_imputed = impute_data(df_train_3)
anova3_p, anova3_f = full_anova(df_train_3_imputed.iloc[:, :-1], df_train_3_imputed.iloc[:, -1])

df_train_4_imputed = impute_data(df_train_4)
anova4_p, anova4_f = full_anova(df_train_4_imputed.iloc[:, :-1], df_train_4_imputed.iloc[:, -1])


df_train_5_imputed = impute_data(df_train_5)
anova5_p, anova5_f = full_anova(df_train_5_imputed.iloc[:, :-1], df_train_5_imputed.iloc[:, -1])

df_train_6_imputed = impute_data(df_train_6)
anova6_p, anova6_f = full_anova(df_train_6_imputed.iloc[:, :-1], df_train_6_imputed.iloc[:, -1])


df_train_7_imputed = impute_data(df_train_7)
anova7_p, anova7_f = full_anova(df_train_7_imputed.iloc[:, :-1], df_train_7_imputed.iloc[:, -1])


df_train_8_imputed = impute_data(df_train_8)
anova8_p, anova8_f = full_anova(df_train_8_imputed.iloc[:, :-1], df_train_8_imputed.iloc[:, -1])


df_train_9_imputed = impute_data(df_train_9)
anova9_p, anova9_f = full_anova(df_train_9_imputed.iloc[:, :-1], df_train_9_imputed.iloc[:, -1])


df_train_10_imputed = impute_data(df_train_10)
anova10_p, anova10_f = full_anova(df_train_10_imputed.iloc[:, :-1], df_train_10_imputed.iloc[:, -1])


df_train_11_imputed = impute_data(df_train_11)
anova11_p, anova11_f = full_anova(df_train_11_imputed.iloc[:, :-1], df_train_11_imputed.iloc[:, -1])


df_train_12_imputed = impute_data(df_train_12)
anova12_p, anova12_f = full_anova(df_train_12_imputed.iloc[:, :-1], df_train_12_imputed.iloc[:, -1])


#concatenate all p-values
all_anova_p_vals = {**anova0_p, **anova1_p, **anova2_p, **anova3_p, **anova4_p, **anova5_p, **anova6_p, **anova7_p, **anova8_p, **anova9_p, **anova10_p, **anova11_p, **anova12_p}
sorted_dict = dict(sorted(all_anova_p_vals.items(), key=lambda item: item[1], reverse=False))
df_anova_p = pd.DataFrame({'Keys': list(sorted_dict.keys()), 'Values': list(sorted_dict.values())})
df_anova_p.to_csv('anova_results_TCGA_collection.csv')

#concatenate all f-values
all_anova_f_vals = {**anova0_f, **anova1_f, **anova2_f, **anova3_f, **anova4_f, **anova5_f, **anova6_f, **anova7_f, **anova8_f, **anova9_f, **anova10_f, **anova11_f, **anova12_f}
sorted_dict_2 = dict(sorted(all_anova_f_vals.items(), key=lambda item: item[1], reverse=True))
df_anova_f = pd.DataFrame({'Keys': list(sorted_dict_2.keys()), 'Values': list(sorted_dict_2.values())})
df_anova_f.to_csv('anova_results_TCGA_collection_F_vals.csv')

In [ ]:
p_vals_file = pd.read_csv("C:\\Users\\tsoni\\Downloads\\anova_results_corrected_TCGA_collection (1).csv")
f_vals_file = pd.read_csv("C:\\Users\\tsoni\\Downloads\\anova_results_corrected_TCGA_collection_F_vals (1).csv")

Get top features from ANOVA & Elastic Net

In [ ]:
def elastic_net(X_train, y_train, X_test, y_test, alpha, l1_ratio, bootstrap_threshold):
  '''
  logistic regression with elastic net regularization
  @X_train: training dataset
  @y_train: corresponding labels for training dataset
  @X_test: test dataset
  @y_test: corresponding labels for test dataset
  @alpha: learning rate
  @l1_ratio: elastic net mixing parameter
  @bootstrap_threshold: threshold to determine which features should be selected as the most important features based on their weightage rankings in the different bootstrapping runs (default: 2 out of 3)
  '''


  all_feature_indices = []
  test_accuracy = 0
  test_precision = 0
  test_recall = 0
  test_f1_score = 0

  # create and run model for each bootstrapping run
  for i in range(3):
    X_bootstrap, y_bootstrap = resample(X_train, y_train, replace=True)
    X_bootstrap = X_bootstrap.drop(X_bootstrap.columns[0], axis=1)

    logreg = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=l1_ratio, C=1/alpha, multi_class='multinomial')
    logreg.fit(X_bootstrap, y_bootstrap)

    #select highest weight features
    feature_selector = SelectFromModel(logreg, prefit=True)
    mask = feature_selector.get_support()
    indices = np.where(mask)[0]
    all_feature_indices.extend(indices)

    #get model performance
    accuracy, precision, recall, f1_score, classification_rep = get_performance(X_test, y_test, logreg)
    test_accuracy += accuracy
    test_precision += precision
    test_recall += recall
    test_f1_score += f1_score

  #get average testing dataset performance across the bootstrapped runs
  test_accuracy /= 3
  test_precision /= 3
  test_recall /= 3
  test_f1_score /= 3

  #average test performance
  test_performance = [test_accuracy, test_precision, test_recall, test_f1_score]

  #obtain most important features based on their weights (compared to the threshold parameter)
  final_feature_indices = list(set([feature for feature in all_feature_indices if all_feature_indices.count(feature) >= bootstrap_threshold]))
  final_feature_names = X_bootstrap.columns[final_feature_indices]

  return final_feature_names, test_performance

In [ ]:
def get_performance(X_test, y_test, model):

    '''
    get the performance of a logistic regression or XGBoost model on the test dataset
    @X_test: test dataset
    @y_test: corresponding labels for test dataset
    @model: logistic regression or XGBoost model
    '''

    X_test = X_test.drop(X_test.columns[0], axis=1)
    y_test_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred, average='weighted')
    recall = recall_score(y_test, y_test_pred, average='weighted')
    f1 = f1_score(y_test, y_test_pred, average='weighted')
    classification_rep = classification_report(y_test, y_test_pred)

    return accuracy, precision, recall, f1, classification_rep

In [ ]:
def get_correlated_features(df, corr_threshold):

    '''
    construct a correlation matrix with the features to identify 1 feature from each pair that has a high Pearson Correlation Coefficient (PCC)
    @df: dataframe (rows = training examples, columns = features) from which to remove redundant features
    @corr_threshold: threshold for PCC
    '''

    label_encoder = LabelEncoder()
    X_temp = df.iloc[:, 1:-1]

    y_temp = label_encoder.fit_transform(df.iloc[:, -1])
    decoded_y_p_vals = label_encoder.inverse_transform(y_temp)

    #construct correlation matrix
    correlation_matrix = X_temp.corr()

    highly_correlated_features = set()
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            #remove a feature from each pair if the correlation is greater than the threshold
            if abs(correlation_matrix.iloc[i, j]) >= corr_threshold:
                colname = correlation_matrix.columns[i]
                highly_correlated_features.add(colname)


    #return the features that are highly correlated
    return list(highly_correlated_features)

In [ ]:
def complete_workflow(p_vals, f_vals, anova_features_to_extract, corr_threshold, vif, alpha, l1_ratio, bootstrap_threshold):

    '''
    complete workflow to get the top features based on ANOVA & elastic net with logistic regression
    '''

    p_vals_top = p_vals["Keys"].tolist()[:anova_features_to_extract]
    f_vals_top = {}

    for feature in p_vals_top:
        filtered_df = f_vals[f_vals['Keys'] == feature]
        # Get the third value of the filtered row(s)
        f_val = filtered_df.iloc[0, 2]
        f_vals_top[feature] = f_val

    f_vals_top = {k: v for k, v in sorted(f_vals_top.items(), key=lambda item: item[1], reverse=True)}

    for key, value in f_vals_top.items():
        print(f"{key}: {value}")

    df_anova_new = pd.DataFrame({'CpG site': list(f_vals_top.keys()), 'F-statistic': list(f_vals_top.values())})
    df_anova_new.to_csv('ANOVA Results for Methylation Data with ' + str(anova_features_to_extract) + ' Features.csv')

    desired_features = list(f_vals_top.keys())

    df_elastic_net_train = df_train_0_imputed.copy()
    selected_genes = [gene for gene in desired_features if gene in df_train_0_imputed.columns]
    first_column = df_train_0_imputed.iloc[:, 0]
    last_column = df_train_0_imputed.iloc[:, -1]
    df_elastic_net_train = pd.concat([first_column, df_train_0_imputed[selected_genes]], axis=1)

    #1
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_1_imputed[[gene for gene in desired_features if gene in df_train_1_imputed.columns]]], axis=1)

    #2
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_2_imputed[[gene for gene in desired_features if gene in df_train_2_imputed.columns]]], axis=1)

    #3
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_3_imputed[[gene for gene in desired_features if gene in df_train_3_imputed.columns]]], axis=1)

    #4
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_4_imputed[[gene for gene in desired_features if gene in df_train_4_imputed.columns]]], axis=1)

    #5
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_5_imputed[[gene for gene in desired_features if gene in df_train_5_imputed.columns]]], axis=1)

    #6
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_6_imputed[[gene for gene in desired_features if gene in df_train_6_imputed.columns]]], axis=1)

    #7
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_7_imputed[[gene for gene in desired_features if gene in df_train_7_imputed.columns]]], axis=1)

    #8
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_8_imputed[[gene for gene in desired_features if gene in df_train_8_imputed.columns]]], axis=1)

    #9
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_9_imputed[[gene for gene in desired_features if gene in df_train_9_imputed.columns]]], axis=1)

    #10
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_10_imputed[[gene for gene in desired_features if gene in df_train_10_imputed.columns]]], axis=1)

    #11
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_11_imputed[[gene for gene in desired_features if gene in df_train_11_imputed.columns]]], axis=1)

    #12
    df_elastic_net_train = pd.concat([df_elastic_net_train, df_train_12_imputed[[gene for gene in desired_features if gene in df_train_12_imputed.columns]]], axis=1)

    df_elastic_net_train = pd.concat([df_elastic_net_train, last_column], axis=1)

    #Elastic net testing set
    df_elastic_net_test = df_test_0_imputed.copy()

    selected_genes = [gene for gene in desired_features if gene in df_test_0_imputed.columns]
    first_column = df_test_0_imputed.iloc[:, 0]
    last_column = df_test_0_imputed.iloc[:, -1]

    df_elastic_net_test = pd.concat([first_column, df_test_0_imputed[selected_genes]], axis=1)

    #1
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_1_imputed[[gene for gene in desired_features if gene in df_test_1_imputed.columns]]], axis=1)

    #2
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_2_imputed[[gene for gene in desired_features if gene in df_test_2_imputed.columns]]], axis=1)

    #3
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_3_imputed[[gene for gene in desired_features if gene in df_test_3_imputed.columns]]], axis=1)

    #4
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_4_imputed[[gene for gene in desired_features if gene in df_test_4_imputed.columns]]], axis=1)

    #5
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_5_imputed[[gene for gene in desired_features if gene in df_test_5_imputed.columns]]], axis=1)

    #6
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_6_imputed[[gene for gene in desired_features if gene in df_test_6_imputed.columns]]], axis=1)

    #7
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_7_imputed[[gene for gene in desired_features if gene in df_test_7_imputed.columns]]], axis=1)

    #8
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_8_imputed[[gene for gene in desired_features if gene in df_test_8_imputed.columns]]], axis=1)

    #9
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_9_imputed[[gene for gene in desired_features if gene in df_test_9_imputed.columns]]], axis=1)

    #10
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_10_imputed[[gene for gene in desired_features if gene in df_test_10_imputed.columns]]], axis=1)

    #11
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_11_imputed[[gene for gene in desired_features if gene in df_test_11_imputed.columns]]], axis=1)

    #12
    df_elastic_net_test = pd.concat([df_elastic_net_test, df_test_12_imputed[[gene for gene in desired_features if gene in df_test_12_imputed.columns]]], axis=1)


    df_elastic_net_test = pd.concat([df_elastic_net_test, last_column], axis=1)

    #remove highly correlated features
    cols_to_drop_corr, y_train_encoded, y_train_decoded = get_correlated_features(df_elastic_net_train, corr_threshold)
    df_elastic_net_train = df_elastic_net_train.drop(columns=cols_to_drop_corr)
    df_elastic_net_test = df_elastic_net_test.drop(columns=cols_to_drop_corr)

    #run logistic regression with elastic net regularization
    final_features, test_performance = elastic_net(df_elastic_net_train.iloc[:, :-1], df_elastic_net_train.iloc[:, -1], df_elastic_net_test.iloc[:, :-1], df_elastic_net_test.iloc[:, -1], alpha, l1_ratio, bootstrap_threshold, vif)

    #return the dataset containing only the final features
    return df_elastic_net_train, df_elastic_net_test, final_features, test_performance

In [ ]:
#get top 160 methylation features
df_train_1400, df_test_1400, final_features_1400, test_performance_1400 = complete_workflow(p_vals_file, f_vals_file, 1400, 0.7, False, 0.01, 0.5, 3) #df_train_1400 is saved as a CSV

#methylation features obtained in the original run used in the paper:
final_features_1400 = ['cg06880930', 'cg02505676', 'cg06401414', 'cg10333400', 'cg11222217', 'cg10244976', 'cg02978297', 'cg24705939', 'cg25664220', 'cg07034850', 'cg16276982', 'cg10970500', 'cg04874795', 'cg25470758', 'cg27570661', 'cg03612357', 'cg01889070', 'cg03400374', 'cg00403616', 'cg06379192', 'cg14274357', 'cg10796749', 'cg03348461', 'cg10128511', 'cg10626816', 'cg00576689', 'cg00114029', 'cg25350011', 'cg25115537', 'cg16497661', 'cg06758255', 'cg01980591', 'cg22433862', 'cg03924164', 'cg23479730', 'cg09259081', 'cg25196088', 'cg24615528', 'cg18768784', 'cg18630178', 'cg16145216', 'cg09918657', 'cg20207219', 'cg15707833', 'cg18383585', 'cg26292910', 'cg01602345', 'cg01574481', 'cg25164490', 'cg25286333', 'cg00786761', 'cg05317714', 'cg15580417', 'cg27165794', 'cg07565228', 'cg05965745', 'cg25408237', 'cg26022183', 'cg24925929', 'cg12121643', 'cg00397635', 'cg09112081', 'cg09964625', 'cg18990313', 'cg08754124', 'cg24170085', 'cg17588293', 'cg18294691', 'cg22940848', 'cg26174215', 'cg04321823', 'cg16718276', 'cg06346857', 'cg10775230', 'cg04166042', 'cg02469095', 'cg02602550', 'cg10493186', 'cg08827060', 'cg13225830', 'cg11692307', 'cg23072161', 'cg04599341', 'cg00152577', 'cg06330323', 'cg05517610', 'cg02893344', 'cg00128386', 'cg26884027', 'cg02533339', 'cg16318112', 'cg26490274', 'cg27302190', 'cg14615440', 'cg05716270', 'cg23394673', 'cg08664849', 'cg06005098', 'cg09984392', 'cg05441854', 'cg01822130', 'cg07424912', 'cg02143404', 'cg01207684', 'cg05637536', 'cg07969676', 'cg03579904', 'cg10196558', 'cg04767756', 'cg26109154', 'cg09885505', 'cg20790030', 'cg05277122', 'cg10964944', 'cg17534540', 'cg11594731', 'cg15997512', 'cg16748643', 'cg09792881', 'cg11739633', 'cg23012310', 'cg11504517', 'cg03969696', 'cg00697672', 'cg04719903', 'cg07211044', 'cg26332740', 'cg08428188', 'cg04912843', 'cg24541835', 'cg09899868', 'cg27546066', 'cg06323896', 'cg06211893', 'cg11807529', 'cg17411994', 'cg19827182', 'cg00789143', 'cg01466330', 'cg24962827', 'cg09238801', 'cg12435415', 'cg11789612', 'cg00805880', 'cg07868599', 'cg19148440', 'cg25930780', 'cg12396523', 'cg06717750', 'cg06913337', 'cg14480530', 'cg09618015', 'cg05789704', 'cg19962116', 'cg18879590', 'cg14659662', 'cg19113686', 'cg10620434', 'cg09391949', 'cg24153003']

Get CpG sites from Annotation File

In [ ]:
df_sesame = pd.read_csv("C:\\Users\\tsoni\\Downloads\\SESAME annotation.tsv", delimiter='\t', usecols=[4, 5])

In [ ]:
def get_genes(CpG_sites, annotation_file_df):

    '''
    get gene names from CpG sites based on the annotation file used (from TCGA)
    @CpG_sites: top features obtained
    @annotation_file_df: annotation file from TCGA
    '''

    cpg_gene_dict = {}
    cpg_genome_dict = {}

    for site in CpG_sites:
        rows_with_site = annotation_file_df[annotation_file_df['probeID'] == site]

        value = rows_with_site.iloc[0, 1]

        if value is not None and not (isinstance(value, float) and np.isnan(value)):
            cpg_gene_dict[site] = value
        else:
            cpg_gene_dict[site] = site


    return cpg_gene_dict

In [ ]:
def get_gene(site, annotation_file_df):
    '''
    get gene name from CpG site based on the annotation file used (from TCGA)
    @site: desired CpG site
    @annotation_file_df: annotation file from TCGA
    '''

    rows_with_site = annotation_file_df[annotation_file_df['probeID'] == site]

    value = rows_with_site.iloc[0, 1]

    if value is not None and not (isinstance(value, float) and np.isnan(value)):
        return value
    else:
        return site

In [ ]:
cpg_gene_dict_1400 = get_genes(final_features_1400, df_sesame)

#in the original run used in the paper, gene dict (key: CpG site, value: corresponding gene name):
cpg_gene_dict_1400 = {'cg06880930': 'CPNE2', 'cg02505676': 'ACSF3', 'cg06401414': 'AGO2', 'cg10333400': 'AL023882.1', 'cg11222217': 'CASZ1', 'cg10244976': 'LMF1', 'cg02978297': 'BCAR3;BCAR3-AS1', 'cg24705939': 'RBPMS', 'cg25664220': 'AC097369.4', 'cg07034850': 'ID3', 'cg16276982': 'AC022613.1', 'cg10970500': 'PLEKHG5', 'cg04874795': nan, 'cg25470758': nan, 'cg27570661': 'ZFHX3', 'cg03612357': 'AL391628.1;COQ8A', 'cg01889070': 'AC120498.4', 'cg03400374': 'KCNAB2', 'cg00403616': 'ERICH1', 'cg06379192': 'PRDM16', 'cg14274357': 'AKT1', 'cg10796749': 'RBP7', 'cg03348461': 'CAMTA1', 'cg10128511': 'ST3GAL6;ST3GAL6-AS1', 'cg10626816': nan, 'cg00576689': nan, 'cg00114029': 'DLGAP3', 'cg25350011': 'ANGPT2;MCPH1', 'cg25115537': 'ZHX2', 'cg16497661': 'CKB', 'cg06758255': 'TOX3', 'cg01980591': 'DLGAP2', 'cg22433862': 'AL445490.1;CHMP1AP1', 'cg03924164': nan, 'cg23479730': 'BCL11B', 'cg09259081': 'MEAK7', 'cg25196088': 'CAMTA1', 'cg24615528': 'LAMP3', 'cg18768784': 'AC112504.2;GRK7', 'cg18630178': 'RIN3', 'cg16145216': 'HIVEP3', 'cg09918657': 'MIR193B;MIR193BHG', 'cg20207219': 'C8orf34;C8orf34-AS1', 'cg15707833': 'ASAP1', 'cg18383585': 'NOS1AP', 'cg26292910': 'IP6K2', 'cg01602345': 'PRDM16', 'cg01574481': 'REC8', 'cg25164490': 'SPSB4', 'cg25286333': 'RNF207;RPL22', 'cg00786761': nan, 'cg05317714': 'KIFC2', 'cg15580417': 'KCNJ9', 'cg27165794': 'PNMA1', 'cg07565228': 'LDHD', 'cg05965745': 'PRDM16', 'cg25408237': 'TMEM201', 'cg26022183': 'PAQR6', 'cg24925929': 'SEMA5B', 'cg12121643': 'DGKG', 'cg00397635': 'ZYG11A', 'cg09112081': nan, 'cg09964625': 'TSNARE1', 'cg18990313': 'AC108863.1;HTRA4', 'cg08754124': nan, 'cg24170085': 'SH3BP5', 'cg17588293': 'ZBTB7B', 'cg18294691': 'TTC6', 'cg22940848': 'SKI', 'cg26174215': nan, 'cg04321823': nan, 'cg16718276': 'FAAP20', 'cg06346857': 'ALPL', 'cg10775230': 'TMIE', 'cg04166042': nan, 'cg02469095': 'OSR2', 'cg02602550': 'AC109462.1;IRX6', 'cg10493186': 'PRDM16', 'cg08827060': nan, 'cg13225830': nan, 'cg11692307': 'AL139246.1;AL139246.4;PLCH2', 'cg23072161': nan, 'cg04599341': 'NOXO1;TBL3', 'cg00152577': nan, 'cg06330323': 'TSC2', 'cg05517610': 'AC025839.1', 'cg02893344': 'GSE1', 'cg00128386': 'GSE1', 'cg26884027': 'PTPRU', 'cg02533339': 'MIR193BHG;MIR365A', 'cg16318112': 'AGRN', 'cg26490274': 'GLIS2;PAM16', 'cg27302190': 'C1orf159', 'cg14615440': 'KAZN', 'cg05716270': 'C1QTNF8', 'cg23394673': 'TNFRSF4', 'cg08664849': 'AC093525.2;TBC1D24', 'cg06005098': nan, 'cg09984392': 'AC009908.1;SQLE', 'cg05441854': 'GSE1', 'cg01822130': 'AC109782.1;EPHA6', 'cg07424912': 'SLC38A7', 'cg02143404': 'GSE1', 'cg01207684': 'ADCY9', 'cg05637536': 'SHE;TDRD10', 'cg07969676': 'AC105001.1;AC105001.2', 'cg03579904': 'KLHDC4', 'cg10196558': 'AL691442.1', 'cg04767756': 'EMP2', 'cg26109154': 'KIF26B', 'cg09885505': 'ZFHX3', 'cg20790030': 'AL008733.1;PRDM16', 'cg05277122': nan, 'cg10964944': nan, 'cg17534540': 'HMCES', 'cg11594731': 'PTH1R', 'cg15997512': 'GFRA2', 'cg16748643': 'TPSP2', 'cg09792881': 'DMRTA2', 'cg11739633': 'MEF2D', 'cg23012310': nan, 'cg11504517': 'TP73;WRAP73', 'cg03969696': 'AC242988.1;CA14', 'cg00697672': nan, 'cg04719903': 'C1QTNF12', 'cg07211044': 'AC090152.1;TOX', 'cg26332740': 'WNT3A', 'cg08428188': 'GSE1', 'cg04912843': 'GIPC2', 'cg24541835': 'DHRS3', 'cg09899868': nan, 'cg27546066': 'WNT3A', 'cg06323896': 'GSE1', 'cg06211893': 'DNM3', 'cg11807529': 'ARHGEF10L', 'cg17411994': 'LINC01137', 'cg19827182': 'ECE1', 'cg00789143': 'LINC02325', 'cg01466330': 'AC007991.3;IDO2', 'cg24962827': 'THRB', 'cg09238801': 'PNMA1', 'cg12435415': 'AC068631.1;GPS2P2;RNU6-1105P', 'cg11789612': 'ITPKA', 'cg00805880': 'PPFIA4', 'cg07868599': 'BICDL2', 'cg19148440': 'IL17RE', 'cg25930780': 'LTF', 'cg12396523': 'TRPS1', 'cg06717750': 'RAB40C', 'cg06913337': 'AC116552.1;ZFPM1', 'cg14480530': 'DENND3', 'cg09618015': 'MIR1224;VWA5B2', 'cg05789704': 'ADAM32', 'cg19962116': 'AC010210.1;NUDT16P1', 'cg18879590': 'GPR137B', 'cg14659662': 'GLIS1', 'cg19113686': 'AC084198.2;RPL24', 'cg10620434': nan, 'cg09391949': 'AC010536.1;JPH3;KLHDC4', 'cg24153003': 'AL589765.4;MRPL9;OAZ3'}

XGBoost

In [ ]:
def recursive_feature_elimination(X_train, y_train, X_test, y_test, num_features_to_select):

    '''
    implement XGBoost with recursive feature elimination to select the most important DNA methylation & gene expression features
    @X_train: training dataset (rows = training examples, columns = features)
    @y_train: corresponding labels for training dataset
    @num_features_to_select: the number of top features to select (i.e. most predictive features)
    '''

    clf = xgb.XGBClassifier(objective='multi:softmax', num_class=6, random_state=42, eval_metric='mlogloss')

    rfe = RFE(estimator=clf, n_features_to_select=num_features_to_select)

    rfe.fit(X_train, y_train)

    # get selected features
    selected_features = rfe.support_
    selected_feature_indices = np.where(selected_features)[0]
    print(len(selected_feature_indices))

    # train the model on the selected features
    print(X_train.iloc[:, selected_feature_indices])
    clf.fit(X_train.iloc[:, selected_feature_indices], y_train)

    #get model performance
    get_XGBoost_performance(X_test.iloc[:, selected_feature_indices], y_test, clf)

    return X_train.columns[selected_feature_indices]

In [ ]:
#confusion matrices for these runs are shown in the paper

X_train_XGBoost = df_train_1400[final_features_1400]
y_train_XGBoost = df_train_1400.iloc[:, -1]

label_encoder = LabelEncoder()
y_train_XGBoost = label_encoder.fit_transform(y_train_XGBoost)

X_test_XGBoost = df_test_1400[final_features_1400]
y_test_XGBoost = df_test_1400.iloc[:, -1]
y_test_XGBoost = label_encoder.transform(y_test_XGBoost)

#1 feature: CPNE2
features_XGBoost = recursive_feature_elimination(X_train_XGBoost, y_train_XGBoost, X_test_XGBoost, y_test_XGBoost, 1)

#top 6 features:
features_XGBoost_6 = recursive_feature_elimination(X_train_XGBoost, y_train_XGBoost, X_test_XGBoost, y_test_XGBoost_6, 6)

Plots for methylation levels of genes across subtypes

In [ ]:
def create_plots(df_train_desired_features):

    '''
    create plots for the methylation levels of the selected features across the cancer subtypes, with error bars
    @df_train_desired_features: training dataset with only the desired features (columns) to be plotted
    '''

    bronchus_and_lung, breast, kidney, brain, thyroid_gland, prostate_gland = get_cancer_subtype_lists(df_train_desired_features.iloc[:, :-1], df_train_desired_features.iloc[:, -1])

    feature_names = df_train_desired_features.iloc[:, 1:-1].columns.tolist()

    data = []

    #means for each feature
    data.append(np.mean(bronchus_and_lung, axis=0))
    data.append(np.mean(breast, axis=0))
    data.append(np.mean(kidney, axis=0))
    data.append(np.mean(brain, axis=0))
    data.append(np.mean(thyroid_gland, axis=0))
    data.append(np.mean(prostate_gland, axis=0))

    #each row is 1 feature, each column is a cancer subtype
    data = np.array(data).astype(float).T

    #find the mean of each row (length is number of features)
    means_for_one_feature = np.mean(data, axis=1)

    #find std for each row
    std_for_one_feature = np.std(data, axis=1)
    sem = std_for_one_feature / np.sqrt(data.shape[1])

    num_features = data.shape[0]

    #for each desired feature, make the plot
    for i in range(num_features):
        print("feature: " + feature_names[i])
        print(get_gene(feature_names[i], df_sesame))
        x_labels = ['Lung', 'Breast', 'Kidney', 'Brain', 'Thyroid', 'Prostate']
        plt.bar(x_labels, data[i], capsize=5, color='#A9A9A9')
        plt.errorbar(x_labels, data[i], yerr=2*sem[i], capsize=5, fmt='none', linewidth=5, ecolor='gray')
        plt.yticks([0, 0.5, 1], fontsize=18)
        plt.xticks(fontsize=18, rotation=45)
        plt.xlabel('Cancer Subtype', fontsize=23)
        plt.ylabel('Methylation Level', fontsize=23)
        plt.title(get_gene(feature_names[i], df_sesame), fontsize=25)
        plt.tight_layout()

        plt.show()

Hierarchical Clustering (only on train dataset)

In [ ]:
#create a copy of the dataset with the final features to be used for clustering analysis (get dataset ready for hierarchical clustering)

df_train_1400 = pd.read_csv('C:\\Users\\tsoni\\df_train_1400.csv')
df_clustering = df_train_1400[final_features_1400].copy()
df_clustering.index = df_train_1400.iloc[:, 1]
df_labeled = pd.concat([df_train_1400[final_features_1400].copy(), df_train_1400.iloc[:, -1]], axis=1)
df_labeled.index = df_train_1400.iloc[:, 1]

In [ ]:
from scipy.cluster.hierarchy import fcluster

linkage_matrix_row_new = clustermap.dendrogram_row.linkage
linkage_matrix_col_new = clustermap.dendrogram_col.linkage

height_row = 35
height_col = 50 #50: 6 clusters
row_clusters_new = fcluster(linkage_matrix_row_new, t=height_row, criterion='distance')
col_clusters_new = fcluster(linkage_matrix_col_new, t=height_col, criterion='distance')

# count the number of unique clusters
num_row_clusters_new = len(np.unique(row_clusters_new))
num_col_clusters_new = len(np.unique(col_clusters_new))

# map clusters back to original row and column labels
row_labels_new = df_clustering.index
col_labels_new = df_clustering.columns

row_clusters_dict_new = dict(zip(row_labels_new, row_clusters_new))
col_clusters_dict_new = dict(zip(col_labels_new, col_clusters_new))

In [ ]:
cluster_1_new = list(get_genes(get_col_cluster_features(1, df_clustering, col_clusters_new).tolist(), df_sesame).values())
print(cluster_1_new)
print(len(cluster_1_new))
cluster_2_new = list(get_genes(get_col_cluster_features(2, df_clustering, col_clusters_new).tolist(), df_sesame).values())
print(cluster_2_new)
print(len(cluster_2_new))
cluster_3_new = list(get_genes(get_col_cluster_features(3, df_clustering, col_clusters_new).tolist(), df_sesame).values())
print(cluster_3_new)
print(len(cluster_3_new))
cluster_4_new = list(get_genes(get_col_cluster_features(4, df_clustering, col_clusters_new).tolist(), df_sesame).values())
print(cluster_4_new)
print(len(cluster_4_new))
cluster_5_new = list(get_genes(get_col_cluster_features(5, df_clustering, col_clusters_new).tolist(), df_sesame).values())
print(cluster_5_new)
print(len(cluster_5_new))
cluster_6_new = list(get_genes(get_col_cluster_features(6, df_clustering, col_clusters_new).tolist(), df_sesame).values())
print(cluster_6_new)
print(len(cluster_6_new))
cluster_7_new = list(get_genes(get_col_cluster_features(7, df_clustering, col_clusters_new).tolist(), df_sesame).values())
print(cluster_7_new)
print(len(cluster_7_new))

#clusters obtained in the paper:

cluster_1_new = ['AGO2', 'MEAK7', 'ASAP1', 'NOS1AP', 'RNF207;RPL22', 'PNMA1', 'ZYG11A', 'TTC6', 'SKI', 'OSR2', 'cg08827060', 'AC009908.1;SQLE', 'AC105001.1;AC105001.2', 'DMRTA2', 'AC242988.1;CA14', 'WNT3A', 'GIPC2', 'DNM3', 'PNMA1', 'AC068631.1;GPS2P2;RNU6-1105P', 'IL17RE', 'LTF', 'TRPS1', 'GPR137B']
cluster_2_new = ['BCAR3;BCAR3-AS1', 'ID3', 'KCNAB2', 'ST3GAL6;ST3GAL6-AS1', 'DLGAP3', 'LAMP3', 'AC112504.2;GRK7', 'KIFC2', 'KCNJ9', 'PAQR6', 'DGKG', 'cg04321823', 'cg04166042', 'PTPRU', 'KAZN', 'TNFRSF4', 'AC109782.1;EPHA6', 'ADAM32', 'AC010210.1;NUDT16P1', 'GLIS1']
cluster_3_new = ['CAMTA1', 'cg10626816', 'CKB', 'BCL11B', 'cg09112081', 'TMIE', 'AC109462.1;IRX6', 'NOXO1;TBL3', 'cg00152577', 'GSE1', 'SHE;TDRD10', 'AL691442.1', 'HMCES', 'PTH1R', 'MEF2D', 'C1QTNF12', 'AC090152.1;TOX', 'WNT3A', 'THRB', 'ITPKA', 'PPFIA4', 'BICDL2']
cluster_4_new = ['CPNE2', 'CASZ1', 'PLEKHG5', 'ERICH1', 'PRDM16', 'AKT1', 'cg00576689', 'DLGAP2', 'AL445490.1;CHMP1AP1', 'CAMTA1', 'RIN3', 'PRDM16', 'cg00786761', 'LDHD', 'TMEM201', 'TSNARE1', 'cg08754124', 'AL139246.1;AL139246.4;PLCH2', 'TSC2', 'GSE1', 'GLIS2;PAM16', 'AC093525.2;TBC1D24', 'KLHDC4', 'KIF26B', 'AL008733.1;PRDM16', 'cg05277122', 'GFRA2', 'GSE1', 'ARHGEF10L', 'ECE1', 'LINC02325', 'AC007991.3;IDO2', 'RAB40C', 'MIR1224;VWA5B2', 'cg10620434', 'AC010536.1;JPH3;KLHDC4']
cluster_5_new = ['LMF1', 'RBPMS', 'ANGPT2;MCPH1', 'ZHX2', 'TOX3', 'IP6K2', 'REC8', 'SPSB4', 'PRDM16', 'GSE1', 'AGRN', 'GSE1', 'ADCY9', 'cg00697672', 'AC116552.1;ZFPM1']
cluster_6_new = ['ACSF3', 'AL023882.1', 'AC097369.4', 'AC022613.1', 'cg04874795', 'cg25470758', 'ZFHX3', 'AL391628.1;COQ8A', 'AC120498.4', 'RBP7', 'cg03924164', 'HIVEP3', 'MIR193B;MIR193BHG', 'C8orf34;C8orf34-AS1', 'SEMA5B', 'AC108863.1;HTRA4', 'SH3BP5', 'ZBTB7B', 'cg26174215', 'FAAP20', 'ALPL', 'PRDM16', 'cg13225830', 'cg23072161', 'AC025839.1', 'MIR193BHG;MIR365A', 'C1orf159', 'C1QTNF8', 'cg06005098', 'SLC38A7', 'EMP2', 'ZFHX3', 'cg10964944', 'TPSP2', 'cg23012310', 'TP73;WRAP73', 'GSE1', 'DHRS3', 'cg09899868', 'LINC01137', 'DENND3', 'AC084198.2;RPL24', 'AL589765.4;MRPL9;OAZ3']

Gene Expression Workflow

Data preprocessing

In [ ]:
def logistic_regression_normalization(df):

    '''
    conduct logistic regression normalization on the dataset
    @df: gene expression training or test dataset

    '''
    first_col = df.iloc[:, 0]
    last_col = df.iloc[:, -1]

    df_normalized = df.iloc[:, 1:-1].astype(float)

    # apply logistic regression normalization for each column (excluding first and last)
    df_normalized = 1 / (1 + (np.exp(-df_normalized)))

    # combine the normalized columns with the first and last columns
    df_normalized = pd.concat([first_col, df_normalized, last_col], axis=1)

    return df_normalized

In [ ]:
df_exp_train = pd.read_csv("C:\\Users\\tsoni\\Downloads\\gene_exp_cleaned_training_data_all_genes.csv")
df_exp_test = pd.read_csv("C:\\Users\\tsoni\\Downloads\\gene_exp_cleaned_testing_data_all_genes.csv")

In [ ]:
df_exp_train_copy = df_exp_train.copy()
df_exp_test_copy = df_exp_test.copy()

df_exp_train_copy = df_exp_train_copy.iloc[2:, 1:]
df_exp_test_copy = df_exp_test_copy.iloc[2:, 1:]

In [ ]:
df_exp_train_copy = logistic_regression_normalization(df_exp_train_copy)
df_exp_test_copy = logistic_regression_normalization(df_exp_test_copy)

Run entire gene expression workflow (using same functions as those used for methylation workflow)

In [ ]:
anova_p, anova_f = full_anova(df_exp_train_copy.iloc[:, :-1], df_exp_train_copy.iloc[:, -1])

sorted_dict_1 = dict(sorted(anova_p.items(), key=lambda item: item[1], reverse=False))
df_anova_p = pd.DataFrame({'Keys': list(sorted_dict_1.keys()), 'Values': list(sorted_dict_1.values())})
df_anova_p.to_csv('all_genes_anova_results_p_vals_exp.csv')

sorted_dict_2 = dict(sorted(anova_f.items(), key=lambda item: item[1], reverse=True))
df_anova_f = pd.DataFrame({'Keys': list(sorted_dict_2.keys()), 'Values': list(sorted_dict_2.values())})
df_anova_f.to_csv('all_genes_anova_results_f_vals_exp.csv')

In [ ]:
#files created
p_vals = pd.read_csv('/Users/aashnasoni/all_genes_anova_results_p_vals_exp_CORRECT.csv')
f_vals = pd.read_csv('/Users/aashnasoni/all_genes_anova_results_f_vals_exp_CORRECT.csv')

In [ ]:
#800, threshold of 0.7 - 160 features
final_features_800, test_performance_800, final_feature_coefficients_800 = full_workflow(df_exp_train, df_exp_test, 800, p_vals, f_vals, 0.7, False, 0.01, 0.5, 3)

#for reference, in the paper:
final_features_800 = ['ENSG00000083307.12', 'ENSG00000101951.17', 'ENSG00000104941.8', 'ENSG00000111816.8', 'ENSG00000100557.10', 'ENSG00000110484.7', 'ENSG00000010379.16', 'ENSG00000047936.11', 'ENSG00000070526.15', 'ENSG00000115705.22', 'ENSG00000125285.6', 'ENSG00000078399.19', 'ENSG00000124215.17', 'ENSG00000096088.16', 'ENSG00000121957.15', 'ENSG00000118526.7', 'ENSG00000100146.18', 'ENSG00000037965.6', 'ENSG00000102313.9', 'ENSG00000120075.5', 'ENSG00000005187.12', 'ENSG00000064787.13', 'ENSG00000115507.10', 'ENSG00000122966.17', 'ENSG00000109956.13', 'ENSG00000115361.8', 'ENSG00000110427.17', 'ENSG00000124302.13', 'ENSG00000100253.13', 'ENSG00000110693.18', 'ENSG00000114771.14', 'ENSG00000113494.17', 'ENSG00000105695.15', 'ENSG00000124935.4', 'ENSG00000100218.12', 'ENSG00000112246.10', 'ENSG00000111291.8', 'ENSG00000108244.17', 'ENSG00000108242.13', 'ENSG00000118193.12', 'ENSG00000084628.10', 'ENSG00000080293.10', 'ENSG00000105376.5', 'ENSG00000096006.12', 'ENSG00000048540.15', 'ENSG00000113430.10', 'ENSG00000089225.20', 'ENSG00000105737.9', 'ENSG00000105989.10', 'ENSG00000124920.14', 'ENSG00000064195.7', 'ENSG00000043591.6', 'ENSG00000124479.11', 'ENSG00000072657.9', 'ENSG00000119125.17', 'ENSG00000117983.17', 'ENSG00000078295.17', 'ENSG00000123364.5', 'ENSG00000080493.18', 'ENSG00000108878.5', 'ENSG00000116299.17', 'ENSG00000042832.12', 'ENSG00000091513.16', 'ENSG00000119514.7', 'ENSG00000058404.20', 'ENSG00000047648.23', 'ENSG00000109339.24', 'ENSG00000100427.16', 'ENSG00000113946.4', 'ENSG00000124205.18', 'ENSG00000121380.12', 'ENSG00000060718.22', 'ENSG00000058866.15', 'ENSG00000111728.10', 'ENSG00000101098.13', 'ENSG00000103241.7', 'ENSG00000091137.14', 'ENSG00000103316.12', 'ENSG00000068976.14', 'ENSG00000110169.11', 'ENSG00000112232.10', 'ENSG00000124343.14', 'ENSG00000106031.9', 'ENSG00000099958.15', 'ENSG00000100626.17', 'ENSG00000005102.14', 'ENSG00000008196.13', 'ENSG00000084710.14', 'ENSG00000108309.14', 'ENSG00000100170.10', 'ENSG00000099625.13', 'ENSG00000088538.13', 'ENSG00000105852.11', 'ENSG00000079689.14', 'ENSG00000064218.5', 'ENSG00000100285.10', 'ENSG00000101049.17', 'ENSG00000121270.16', 'ENSG00000115598.10', 'ENSG00000124939.6', 'ENSG00000009765.15', 'ENSG00000113763.12', 'ENSG00000009694.13', 'ENSG00000113361.13', 'ENSG00000124249.7', 'ENSG00000121933.19', 'ENSG00000113396.13', 'ENSG00000103460.17', 'ENSG00000099954.18', 'ENSG00000109819.9', 'ENSG00000124875.10', 'ENSG00000108813.11', 'ENSG00000091831.24', 'ENSG00000107105.15', 'ENSG00000087128.10', 'ENSG00000105048.17', 'ENSG00000108602.18', 'ENSG00000064205.10', 'ENSG00000102349.18', 'ENSG00000091622.16', 'ENSG00000078898.7', 'ENSG00000109182.12', 'ENSG00000070731.11', 'ENSG00000111452.13', 'ENSG00000117009.12', 'ENSG00000080031.10', 'ENSG00000108018.15', 'ENSG00000108001.15', 'ENSG00000119946.11', 'ENSG00000095627.9', 'ENSG00000100473.18', 'ENSG00000124429.18', 'ENSG00000007038.11', 'ENSG00000091844.8', 'ENSG00000095932.7', 'ENSG00000104059.4', 'ENSG00000081479.15', 'ENSG00000083782.8', 'ENSG00000050555.19', 'ENSG00000089199.10', 'ENSG00000104435.14', 'ENSG00000082482.14', 'ENSG00000107159.13', 'ENSG00000007908.16', 'ENSG00000124440.16', 'ENSG00000105131.8', 'ENSG00000112175.8', 'ENSG00000122756.15', 'ENSG00000050628.20', 'ENSG00000067048.17', 'ENSG00000123360.12', 'ENSG00000118596.12', 'ENSG00000101842.14', 'ENSG00000082175.15', 'ENSG00000088340.17', 'ENSG00000109906.14', 'ENSG00000026559.14', 'ENSG00000120659.15', 'ENSG00000019186.10', 'ENSG00000077063.11']


In [ ]:
#get the top 6 gene expression features
X_train_XGBoost_6 = df_exp_train_copy[final_features_800]
y_train_XGBoost_6 = df_exp_train_copy.iloc[:, -1]

label_encoder = LabelEncoder()
y_train_XGBoost_6 = label_encoder.fit_transform(y_train_XGBoost_6)

X_test_XGBoost_6 = df_exp_test_copy[final_features_800]
y_test_XGBoost_6 = df_exp_test_copy.iloc[:, -1]
y_test_XGBoost_6 = label_encoder.transform(y_test_XGBoost_6)

features_XGBoost_6 = recursive_feature_elimination(X_train_XGBoost_6, y_train_XGBoost_6, X_test_XGBoost_6, y_test_XGBoost_6, 6)

Get gene IDs (gene name + type)

In [ ]:
import csv

def get_column_values(file_path, column_index, start=None, end=None):
    '''
    used to get the gene IDs, gene names, and gene types from the gene expression annotation file
    '''

    column_values = []

    with open(file_path, 'r') as tsv_file:
        lines = tsv_file.readlines()

    column_values = [line.strip().split('\t')[column_index] for line in lines[start:end]]

    return column_values

In [ ]:
gene_ids = get_column_values('/Users/aashnasoni/Desktop/TestGeneExpDirec/gdc_download_20230801_023132.225603/4df2233b-3bbc-4d20-9abc-2a09b3f37383/eb0f8bc7-5f47-4174-8e85-678bd011bc3c.rna_seq.augmented_star_gene_counts.tsv', 0, 6)
gene_names = get_column_values('/Users/aashnasoni/Desktop/TestGeneExpDirec/gdc_download_20230801_023132.225603/4df2233b-3bbc-4d20-9abc-2a09b3f37383/eb0f8bc7-5f47-4174-8e85-678bd011bc3c.rna_seq.augmented_star_gene_counts.tsv', 1, 6)
gene_types = get_column_values('/Users/aashnasoni/Desktop/TestGeneExpDirec/gdc_download_20230801_023132.225603/4df2233b-3bbc-4d20-9abc-2a09b3f37383/eb0f8bc7-5f47-4174-8e85-678bd011bc3c.rna_seq.augmented_star_gene_counts.tsv', 2, 6)

In [ ]:
result_dict = {key : [value2, value3] for key, value2, value3 in zip(gene_ids, gene_names, gene_types)}

In [ ]:
def get_gene_description(ensemble_id, result_dict):
    '''
    obtain a detailed description of a gene's function from the annotation file (ex: gene type, protein it encodes)
    '''
    return ', '.join(result_dict[ensemble_id])

In [ ]:
def get_gene_id(gene_description, result_dict):
    '''
    get the gene ID of a gene from its description
    '''
    list_version = gene_description.split(', ')
    for key, value in result_dict.items():
        if value == list_version:
            return key

In [ ]:
def get_gene_ids(gene_descriptions, result_dict):
    '''
    get the gene IDs of a list of genes from their descriptions
    '''
    ids = []
    for desc in gene_descriptions:
        ids.append(get_gene_id(desc, result_dict))
    return ids

Bar plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_plots(df_train_desired_features):

    '''
    obtain a detailed description of a gene's function from the annotation file (ex: gene type, protein it encodes)
    '''

    bronchus_and_lung, breast, kidney, brain, thyroid_gland, prostate_gland = get_cancer_subtype_lists(df_train_desired_features.iloc[:, :-1], df_train_desired_features.iloc[:, -1])

    feature_names = df_train_desired_features.iloc[:, 1:-1].columns.tolist()

    data = []

    #means for each feature
    data.append(np.mean(bronchus_and_lung, axis=0))
    data.append(np.mean(breast, axis=0))
    data.append(np.mean(kidney, axis=0))
    data.append(np.mean(brain, axis=0))
    data.append(np.mean(thyroid_gland, axis=0))
    data.append(np.mean(prostate_gland, axis=0))

    #each row is 1 feature, each column is a cancer subtype
    data = np.array(data).astype(float).T

    #find the mean of each row (length is number of features)
    means_for_one_feature = np.mean(data, axis=1)

    #find std for each row
    std_for_one_feature = np.std(data, axis=1)
    sem = std_for_one_feature / np.sqrt(data.shape[1])

    num_features = data.shape[0]
    #for each desired feature
    for i in range(num_features):
        print("feature: " + feature_names[i])
        x_labels = ['Lung', 'Breast', 'Kidney', 'Brain', 'Thyroid', 'Prostate']
        plt.bar(x_labels, data[i], capsize=5, color='#A9A9A9')
        plt.errorbar(x_labels, data[i], yerr=2*sem[i], capsize=5, fmt='none', linewidth=5, ecolor='gray')
        plt.yticks([0, 0.5, 1], fontsize=18)  # Adjust tick locations and font size
        plt.xticks(fontsize=18, rotation=45)
        plt.xlabel('Cancer Subtype', fontsize=23)
        plt.ylabel('Expression Level', fontsize=23)
        plt.title(get_gene_name(feature_names[i], result_dict), fontsize=25)
        plt.tight_layout()

        plt.show()

In [ ]:
top_6_RFE = pd.concat([df_exp_train_copy.iloc[:, 0], df_exp_train_copy[features_XGBoost_6], df_exp_train_copy.iloc[:, -1]], axis=1)
create_plots(top_6_RFE)

Clustering

In [ ]:
df_train_clustering = df_exp_train_copy[final_features_160].copy()
df_train_clustering.index = df_exp_train_copy.iloc[:, 0]
print(df_train_clustering.shape)

df_labeled = pd.concat([df_exp_train_copy[final_features_160].copy(), df_exp_train_copy.iloc[:, -1]], axis=1)
df_labeled.index = df_exp_train_copy.iloc[:, 0]
print(df_labeled)

In [ ]:
clustermap = sns.clustermap(df_train_clustering, row_cluster=True, col_cluster=True, cmap='mako', method='ward')

In [ ]:
from scipy.cluster.hierarchy import fcluster

linkage_matrix_row_new = clustermap.dendrogram_row.linkage
linkage_matrix_col_new = clustermap.dendrogram_col.linkage

height_row = 27
height_col = 38 #get 6 column clusters
row_clusters_new = fcluster(linkage_matrix_row_new, t=height_row, criterion='distance')
col_clusters_new = fcluster(linkage_matrix_col_new, t=height_col, criterion='distance')

# Count the number of unique clusters
num_row_clusters_new = len(np.unique(row_clusters_new))
num_col_clusters_new = len(np.unique(col_clusters_new))

# Print the results
print(f"Number of row clusters: {num_row_clusters_new}")
print(f"Number of column clusters: {num_col_clusters_new}")

# Map clusters back to original row and column labels
row_labels_new = df_train_clustering.index
col_labels_new = df_train_clustering.columns

row_clusters_dict_new = dict(zip(row_labels_new, row_clusters_new))
col_clusters_dict_new = dict(zip(col_labels_new, col_clusters_new))

In [ ]:
from collections import Counter

#get all the values in a column cluster
def get_col_cluster_features(desired_cluster, df_clustering, col_clusters):
    return df_clustering.columns[col_clusters == desired_cluster]

#get all the values in a row cluster
def get_row_cluster_samples(desired_cluster, df_labeled, row_clusters):
    row_indices_in_target_cluster = np.where(row_clusters == desired_cluster)[0]
    original_row_indices = df_labeled.index[row_indices_in_target_cluster]
    return original_row_indices

def get_cancer_subtype_for_cluster(cluster, df_labeled, row_clusters):
    row_indices_in_target_cluster = np.where(row_clusters == cluster)[0]
    original_row_indices = df_labeled.index[row_indices_in_target_cluster]
    labels = []
    for index in original_row_indices:
        labels.append(df_labeled.loc[index].iloc[-1])
    counter = Counter(labels)
    most_common = counter.most_common(1)
    return most_common

In [ ]:
def get_cancer_subtype_lists(X_bootstrap, y_bootstrap):

  y_bootstrap_values = y_bootstrap.values

  #2D arrays
  bronchus_and_lung = []
  breast = []
  kidney = []
  brain = []
  thyroid_gland = []
  prostate_gland = []

  index = 0

  while index < len(y_bootstrap_values):
    if y_bootstrap_values[index] == "Bronchus and lung":
      bronchus_and_lung.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Breast":
      breast.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Kidney":
      kidney.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Brain":
      brain.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Thyroid gland":
      thyroid_gland.append(X_bootstrap.iloc[index].values[1:])

    if y_bootstrap_values[index] == "Prostate gland":
      prostate_gland.append(X_bootstrap.iloc[index].values[1:])

    index += 1

  #convert to numpy arrays - [meth markers...cancer subtype] for each sample

  bronchus_and_lung = np.array(bronchus_and_lung)
  breast = np.array(breast)
  kidney = np.array(kidney)
  brain = np.array(brain)
  thyroid_gland = np.array(thyroid_gland)
  prostate_gland = np.array(prostate_gland)

  return bronchus_and_lung, breast, kidney, brain, thyroid_gland, prostate_gland

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_plots(df_train_desired_features):

    '''
    create plots for the gene expression levels of the selected features across the cancer subtypes, with error bars
    @df_train_desired_features: training dataset with only the desired features (columns) to be plotted
    '''

    bronchus_and_lung, breast, kidney, brain, thyroid_gland, prostate_gland = get_cancer_subtype_lists(df_train_desired_features.iloc[:, :-1], df_train_desired_features.iloc[:, -1])

    feature_names = df_train_desired_features.iloc[:, 1:-1].columns.tolist()

    data = []

    #means for each feature
    data.append(np.mean(bronchus_and_lung, axis=0))
    data.append(np.mean(breast, axis=0))
    data.append(np.mean(kidney, axis=0))
    data.append(np.mean(brain, axis=0))
    data.append(np.mean(thyroid_gland, axis=0))
    data.append(np.mean(prostate_gland, axis=0))

    #each row is 1 feature, each column is a cancer subtype
    data = np.array(data).astype(float).T

    #find the mean of each row (length is number of features)
    means_for_one_feature = np.mean(data, axis=1)

    #find std for each row
    std_for_one_feature = np.std(data, axis=1)
    sem = std_for_one_feature / np.sqrt(data.shape[1])

    num_features = data.shape[0]
    #for each desired feature
    for i in range(num_features):
        print("feature: " + feature_names[i])
        x_labels = ['Lung', 'Breast', 'Kidney', 'Brain', 'Thyroid', 'Prostate']
        plt.bar(x_labels, data[i], yerr=2*sem[i], capsize=5, align='center', alpha=0.7, ecolor="gray")
        plt.yticks([0, 0.5, 1], fontsize=14)
        plt.xticks(fontsize=14)
        plt.xlabel('Cancer Subtype', fontsize=18)
        plt.ylabel('Methylation Level', fontsize=18)
        plt.tight_layout()

        plt.show()

In [ ]:
#get the gene expression clusters from hierarchical clustering

cluster_1_new = get_col_cluster_features(1, df_train_clustering, col_clusters_new).tolist()
print(cluster_1_new)
print(len(cluster_1_new))
cluster_2_new = get_col_cluster_features(2, df_train_clustering, col_clusters_new).tolist()
print(cluster_2_new)
print(len(cluster_2_new))
cluster_3_new = get_col_cluster_features(3, df_train_clustering, col_clusters_new).tolist()
print(cluster_3_new)
print(len(cluster_3_new))
cluster_4_new = get_col_cluster_features(4, df_train_clustering, col_clusters_new).tolist()
print(cluster_4_new)
print(len(cluster_4_new))
cluster_5_new = get_col_cluster_features(5, df_train_clustering, col_clusters_new).tolist()
print(cluster_5_new)
print(len(cluster_5_new))
cluster_6_new = get_col_cluster_features(6, df_train_clustering, col_clusters_new).tolist()
print(cluster_6_new)
print(len(cluster_6_new))

#cluster values in paper:
cluster_1_new = ['ENSG00000083307.12', 'ENSG00000111816.8', 'ENSG00000110484.7', 'ENSG00000037965.6', 'ENSG00000120075.5', 'ENSG00000005187.12', 'ENSG00000113494.17', 'ENSG00000111291.8', 'ENSG00000108244.17', 'ENSG00000118193.12', 'ENSG00000089225.20', 'ENSG00000105989.10', 'ENSG00000064195.7', 'ENSG00000123364.5', 'ENSG00000119514.7', 'ENSG00000060718.22', 'ENSG00000091137.14', 'ENSG00000124343.14', 'ENSG00000099958.15', 'ENSG00000005102.14', 'ENSG00000008196.13', 'ENSG00000105852.11', 'ENSG00000115598.10', 'ENSG00000124939.6', 'ENSG00000124249.7', 'ENSG00000091831.24', 'ENSG00000105048.17', 'ENSG00000064205.10', 'ENSG00000102349.18', 'ENSG00000070731.11', 'ENSG00000111452.13', 'ENSG00000117009.12', 'ENSG00000108001.15', 'ENSG00000124429.18', 'ENSG00000083782.8', 'ENSG00000007908.16', 'ENSG00000050628.20', 'ENSG00000082175.15', 'ENSG00000120659.15']
cluster_2_new = ['ENSG00000101951.17', 'ENSG00000124215.17', 'ENSG00000096088.16', 'ENSG00000118526.7', 'ENSG00000102313.9', 'ENSG00000115507.10', 'ENSG00000096006.12', 'ENSG00000113430.10', 'ENSG00000121380.12', 'ENSG00000110169.11', 'ENSG00000106031.9', 'ENSG00000121270.16', 'ENSG00000087128.10', 'ENSG00000078898.7', 'ENSG00000109182.12', 'ENSG00000095627.9', 'ENSG00000100473.18', 'ENSG00000007038.11', 'ENSG00000026559.14']
cluster_3_new = ['ENSG00000104941.8', 'ENSG00000047936.11', 'ENSG00000115705.22', 'ENSG00000100253.13', 'ENSG00000114771.14', 'ENSG00000108242.13', 'ENSG00000080293.10', 'ENSG00000117983.17', 'ENSG00000042832.12', 'ENSG00000113946.4', 'ENSG00000064218.5', 'ENSG00000009765.15', 'ENSG00000124875.10', 'ENSG00000108813.11', 'ENSG00000108602.18', 'ENSG00000080031.10', 'ENSG00000107159.13', 'ENSG00000105131.8', 'ENSG00000112175.8', 'ENSG00000101842.14', 'ENSG00000088340.17', 'ENSG00000019186.10']
cluster_4_new = ['ENSG00000100557.10', 'ENSG00000010379.16', 'ENSG00000078399.19', 'ENSG00000115361.8', 'ENSG00000110693.18', 'ENSG00000124935.4', 'ENSG00000100218.12', 'ENSG00000112246.10', 'ENSG00000072657.9', 'ENSG00000119125.17', 'ENSG00000103316.12', 'ENSG00000100170.10', 'ENSG00000079689.14', 'ENSG00000009694.13', 'ENSG00000109819.9', 'ENSG00000119946.11', 'ENSG00000095932.7', 'ENSG00000067048.17']
cluster_5_new = ['ENSG00000070526.15', 'ENSG00000121957.15', 'ENSG00000122966.17', 'ENSG00000109956.13', 'ENSG00000048540.15', 'ENSG00000105737.9', 'ENSG00000124920.14', 'ENSG00000043591.6', 'ENSG00000080493.18', 'ENSG00000116299.17', 'ENSG00000058404.20', 'ENSG00000047648.23', 'ENSG00000109339.24', 'ENSG00000103241.7', 'ENSG00000068976.14', 'ENSG00000100626.17', 'ENSG00000101049.17', 'ENSG00000113361.13', 'ENSG00000121933.19', 'ENSG00000099954.18', 'ENSG00000091622.16', 'ENSG00000091844.8', 'ENSG00000104059.4', 'ENSG00000081479.15', 'ENSG00000050555.19', 'ENSG00000123360.12', 'ENSG00000118596.12', 'ENSG00000109906.14', 'ENSG00000077063.11']
cluster_6_new = ['ENSG00000125285.6', 'ENSG00000100146.18', 'ENSG00000064787.13', 'ENSG00000110427.17', 'ENSG00000124302.13', 'ENSG00000105695.15', 'ENSG00000084628.10', 'ENSG00000105376.5', 'ENSG00000124479.11', 'ENSG00000078295.17', 'ENSG00000108878.5', 'ENSG00000091513.16', 'ENSG00000100427.16', 'ENSG00000124205.18', 'ENSG00000058866.15', 'ENSG00000111728.10', 'ENSG00000101098.13', 'ENSG00000112232.10', 'ENSG00000084710.14', 'ENSG00000108309.14', 'ENSG00000099625.13', 'ENSG00000088538.13', 'ENSG00000100285.10', 'ENSG00000113763.12', 'ENSG00000113396.13', 'ENSG00000103460.17', 'ENSG00000107105.15', 'ENSG00000108018.15', 'ENSG00000089199.10', 'ENSG00000104435.14', 'ENSG00000082482.14', 'ENSG00000124440.16', 'ENSG00000122756.15']
